In [25]:
import pandas as pd
df = pd.DataFrame()
df['1'] = [1,1,1,1,0,1,0,0,0]
df['2'] = [0,1,1,1,3,1,0,0,0]
df['3'] = [0,0,0,0,0,1,1,1,1]
df['4'] = [1,1,0,1,1,0,0,0,0]
df['5'] = [0,0,0,0,0,1,1,1,1]
df['6'] = [0,0,0,0,0,0,1,0,2]

s = pd.Series(['monty','python','sketch','laugh','funny','best','ice_cream','dessert','taste'])
df.set_index([s])

,1,2,3,4,5,6
monty,1,0,0,1,0,0
python,1,1,0,1,0,0
sketch,1,1,0,0,0,0
laugh,1,1,0,1,0,0
funny,0,3,0,1,0,0
best,1,1,1,0,1,0
ice_cream,0,0,1,0,1,1
dessert,0,0,1,0,1,0
taste,0,0,1,0,1,2


In [28]:
df['df'] = [2,3,2,3,2,4,3,2,3]
df['cf'] = [2,3,2,3,4,4,3,2,4]

In [40]:
import numpy as np
df['idf'] = np.log2(6/df['df'])
df['tf_idf1'] = df['1']*df['idf']
df['tf_idf2'] = df['2']*df['idf']
df['tf_idf3'] = df['3']*df['idf']
df['tf_idf4'] = df['4']*df['idf']
df['tf_idf5'] = df['5']*df['idf']
df['tf_idf6'] = df['6']*df['idf']


In [41]:
df

,1,2,3,4,5,6,df,cf,idf,tf_idf1,tf_idf2,tf_idf3,tf_idf4,tf_idf5,tf_idf6
0,1,0,0,1,0,0,2,2,1.584963,1.584963,0.000000,0.000000,1.584963,0.000000,0.0
1,1,1,0,1,0,0,3,3,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0
2,1,1,0,0,0,0,2,2,1.584963,1.584963,1.584963,0.000000,0.000000,0.000000,0.0
3,1,1,0,1,0,0,3,3,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.0
4,0,3,0,1,0,0,2,4,1.584963,0.000000,4.754888,0.000000,1.584963,0.000000,0.0
5,1,1,1,0,1,0,4,4,0.584963,0.584963,0.584963,0.584963,0.000000,0.584963,0.0
6,0,0,1,0,1,1,3,3,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.0
7,0,0,1,0,1,0,2,2,1.584963,0.000000,0.000000,1.584963,0.000000,1.584963,0.0
8,0,0,1,0,1,2,3,4,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.0


In [42]:
# Drill_0

In [43]:
import nltk
from nltk.corpus import gutenberg
nltk.download('punkt')
nltk.download('gutenberg')
import re
from sklearn.model_selection import train_test_split

#reading in the data, this time in the form of paragraphs
emma=gutenberg.paras('austen-emma.txt')
#processing
emma_paras=[]
for paragraph in emma:
    para=paragraph[0]
    #removing the double-dash from all words
    para=[re.sub(r'--','',word) for word in para]
    #Forming each paragraph into a string and adding it to the list of strings.
    emma_paras.append(' '.join(para))

print(emma_paras[0:4])
# print(emma_paras)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Song\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


['[ Emma by Jane Austen 1816 ]', 'VOLUME I', 'CHAPTER I', 'Emma Woodhouse , handsome , clever , and rich , with a comfortable home and happy disposition , seemed to unite some of the best blessings of existence ; and had lived nearly twenty - one years in the world with very little to distress or vex her .']


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test = train_test_split(emma_paras, test_size=0.4, random_state=0)

vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
emma_paras_tfidf=vectorizer.fit_transform(emma_paras)
print("Number of features: %d" % emma_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(emma_paras_tfidf, test_size=0.4, random_state=0)


#Reshapes the vectorizer output into something people can read
X_test_tfidf_csr = X_test_tfidf.tocsr()

#number of paragraphs
n = X_test_tfidf_csr.shape[0]
#A list of dictionaries, one per paragraph
tfidf_bypara = [{} for _ in range(0,n)]
#List of features
terms = vectorizer.get_feature_names()
#for each paragraph, lists the feature words and their tf-idf scores
for i, j in zip(*X_test_tfidf_csr.nonzero()):
    tfidf_bypara[i][terms[j]] = X_test_tfidf_csr[i, j]

#Keep in mind that the log base 2 of 1 is 0, so a tf-idf score of 0 indicates that the word was present once in that sentence.
print('Original sentence:', X_test[5])
print('Tf_idf vector:', tfidf_bypara[5])

Number of features: 1948
Original sentence: " And I am quite serious too , I assure you ," replied Mrs . Elton gaily , " in resolving to be always on the watch , and employing my friends to watch also , that nothing really unexceptionable may pass us ."
Tf_idf vector: {'resolving': 0.3424390305045805, 'unexceptionable': 0.3196409792117937, 'assure': 0.267729777836069, 'friends': 0.2515333740641343, 'quite': 0.1916471918420347, 'replied': 0.20315405638674577, 'watch': 0.62300800500314, 'elton': 0.16874782424809767, 'pass': 0.28445227585106625, 'really': 0.21963852817625326, 'mrs': 0.14780961753993585}


In [47]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

#Our SVD data reducer.  We are going to reduce the feature space from 1379 to 130.
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_test_lsa = lsa.fit_transform(X_test_tfidf)

variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_test_lsa,index=X_test)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])




Percent variance captured by all components: 49.367706222709785
Component 0:
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh !       0.999922
" Oh no !    0.999922
Name: 0, dtype: float64
Component 1:
" Well , Mrs . Weston ," said Emma triumphantly when he left them , " what do you say now to Mr . Knightley ' s marrying Jane Fairfax ?"                                                                                                                                                                                                                                                                                                             0.617841
After tea , Mr . and Mrs . Weston , and Mr . Elton sat down with Mr . Woodhouse to cards .                                                                                                                                  

#### it seems component 1 does not identify similar sentences

In [48]:
vectorizer = TfidfVectorizer(max_df=0.25, # drop words that occur in more than 25% of the paragraphs
                             min_df=3, # only use words that appear at least 3 times
                             stop_words='english', 
                             lowercase=True, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )


#Applying the vectorizer
emma_paras_tfidf=vectorizer.fit_transform(emma_paras)
print("Number of features: %d" % emma_paras_tfidf.get_shape()[1])

#splitting into training and test sets
X_train_tfidf, X_test_tfidf= train_test_split(emma_paras_tfidf, test_size=0.4, random_state=0)

Number of features: 1358


In [49]:
# Keep same number of components
svd= TruncatedSVD(130)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Fit with train data
X_train_lsa = lsa.fit_transform(X_train_tfidf)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Component 0:
" Oh !     0.999234
" Oh !     0.999234
" Oh !     0.999234
" Oh !     0.999234
" Oh !"    0.999234
" Oh !     0.999234
" Oh !     0.999234
" Oh !     0.999234
" Oh !     0.999234
" Oh !"    0.999234
Name: 0, dtype: float64
Component 1:
" You have made her too tall , Emma ," said Mr . Knightley .                                                                                                                0.676339
" There were misunderstandings between them , Emma ; he said so expressly .                                                                                                 0.595230
" You get upon delicate subjects , Emma ," said Mrs . Weston smiling ; " remember that I am here . Mr .                                                                     0.583643
" I do not know what your opinion may be , Mrs . Weston ," said Mr . Knightley , " of this great intimacy between Emma and Harriet Smith , but I think it a bad thing ."    0.573370
Mr . Knightley might quarr

In [50]:
# Reduce to 50 components
svd= TruncatedSVD(50)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

#Looking at what sorts of paragraphs our solution considers similar, for the first five identified topics
paras_by_component=pd.DataFrame(X_train_lsa,index=X_train)
for i in range(5):
    print('Component {}:'.format(i))
    print(paras_by_component.loc[:,i].sort_values(ascending=False)[0:10])

Component 0:
" Oh !     0.999328
" Oh !     0.999328
" Oh !     0.999328
" Oh !     0.999328
" Oh !"    0.999328
" Oh !     0.999328
" Oh !     0.999328
" Oh !     0.999328
" Oh !     0.999328
" Oh !     0.999328
Name: 0, dtype: float64
Component 1:
Mrs . Weston was acting no part , feigning no feelings in all that she said to him in favour of the event . She had been extremely surprized , never more so , than when Emma first opened the affair to her ; but she saw in it only increase of happiness to all , and had no scruple in urging him to the utmost . She had such a regard for Mr . Knightley , as to think he deserved even her dearest Emma ; and it was in every respect so proper , suitable , and unexceptionable a connexion , and in one respect , one point of the highest importance , so peculiarly eligible , so singularly fortunate , that now it seemed as if Emma could not safely have attached herself to any other creature , and that she had herself been the stupidest of beings in not 